In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
os.getcwd()

# Background Global Financial Crisis (GFC)
**[From Wikipedia](https://en.wikipedia.org/wiki/2007%E2%80%932008_financial_crisis):**
- ***Milestone 1: May 19, 2005***: Fund manager Michael Burry closed a credit default swap against subprime mortgage bonds with Deutsche Bank valued at 60 million dollars - the first such CDS. He projected they would become volatile within two years of the low "teaser rate" of the mortgages expiring.

- ***Milestone 2: September 15, 2008***: After the Federal Reserve declined to guarantee its loans as it did for Bear Stearns, the Bankruptcy of Lehman Brothers led to a 504.48-point (4.42%) drop in the DJIA, its worst decline in seven years. To avoid bankruptcy, Merrill Lynch was acquired by Bank of America for $50 billion in a transaction facilitated by the government. Lehman had been in talks to be sold to either Bank of America or Barclays but neither bank wanted to acquire the entire company.

In [ ]:
milestone1 = pd.to_datetime('2005-05-19')  # Michael Burry close a credit default swap against subprime mortgage bonds
milestone2 = pd.to_datetime('2008-09-15')  # Lehman Brothers bankruptcy

## Stability vs Volatility Dynamics

In [ ]:
MARKETS = {
    'UN': 'New York Stock Exchange',
    'UW': 'NASDAQ Global Select Market',
    'LN': 'London Stock Exchange',
    'JT': 'Tokyo Stock Exchange'
}

EXPERIMENTS = [
    {'UN': '003865Y', 'LN': '653851X', 'UW': '643630M', 'JT': '102939K'},
    {'UN': '365374G', 'LN': '655972O', 'UW': '941047N', 'JT': '019647Q'},
    {'UN': '928387S', 'LN': '774892E', 'UW': '960223U', 'JT': '897643T'},
    {'UN': '242769F', 'LN': '963882T', 'UW': '088988V', 'JT': '711534E'},
    {'UN': '565988I', 'LN': '899299O', 'UW': '368353O', 'JT': '668904O'},
    {'UN': '852031O', 'LN': '605850U', 'UW': '021340F', 'JT': '482288K'},
    {'UN': '242914S', 'LN': '565750N', 'UW': '429252I', 'JT': '943908F'},
    {'UN': '709338D', 'LN': '927491K', 'UW': '289657I', 'JT': '902600X'},
    {'UN': '042109Z', 'LN': '715743X', 'UW': '062485K', 'JT': '927218C'},
    {'UN': '855945R', 'LN': '877537K', 'UW': '775312L', 'JT': '637990M'},
    {'UN': '720192X', 'LN': '811006M', 'UW': '739420C', 'JT': '223400A'},
    {'UN': '329900I', 'LN': '971711C', 'UW': '129109Y', 'JT': '864193Z'},
    {'UN': '331719B', 'LN': '927049F', 'UW': '222677X', 'JT': '737397X'},
    {'UN': '992991E', 'LN': '883178B', 'UW': '225521B', 'JT': '091567J'},
    {'UN': '107340Q', 'LN': '258743T', 'UW': '926899H', 'JT': '781892G'},
]
# Experiments with same shifting window
ROLLING_CHUNKS = {
    '4M-4Y': (1, 7, 8, 9),
    # '4M-2Y': (2, 6, 10),
    # '4M-1Y': (3, 4, 5),
    # '2M-2Y': (11, 12, 13),
    # '2M-1Y': (14, 15),
}

ROLLING_FREE_PARAMETERS = {
    1: ('[-0.1, -2.0]', 2000),
    2: ('[-0.1, -2.0]', 2000),
    3: ('[-0.1, -2.0]', 2000),
    4: ('[-0.1, -2.0]', 1000),
    5: ('[-0.1, -2.0]', 500),
    6: ('[-0.1, -1.5]', 2000),
    7: ('[-0.1, -1.5]', 2000),
    8: ('[-0.1, -1.5]', 5000),
    9: ('[-0.1, -1.5]', 500),
    10: ('[-0.1, -1.5]', 500),
    11: ('[-0.1, -2.0]', 500),
    12: ('[-0.0, -1.0]', 1000),
    13: ('[-0.1, -2.0]', 1000),
    14: ('[-0.0, -1.0]', 1000),
    15: ('[-0.1, -2.5]', 1000),
}

In [ ]:
def read_pickle(run_id, market):
    df = pd.read_pickle(f'../data/processed/dynamics/{run_id}.pickle')
    df.index.name = 'Day'
    df['Market'] = market
    return df


def plot_indicators(df):
    axes = df.plot(subplots=True, figsize=(8,16), sharex=False, grid=True)
    for ax in axes:
        ax.axvline('2008-01-01', color='red')
    plt.show()


def plot_indicators_grid(data):
    melted_data = pd.melt(data.reset_index(),
                      id_vars=['Day', 'Market'],
                      value_vars=['Max', 'Peak', 'FWHM'],
                      var_name='indicator', value_name='value')
    # Seaborn FacetGrid plot
    g = sns.relplot(
        data=melted_data,
        x='Day',
        y='value',
        col='Market',
        row='indicator',
        kind='line',
        facet_kws={'sharey': False},
        aspect=1.4,
    )
    # Add vertical lines for milestones
    g.map(lambda **kwargs: plt.axvline(milestone1, color='orange'))
    g.map(lambda **kwargs: plt.axvline(milestone2, color='red'))
    return g

def save_figure(facet, experiment):
    facet.savefig(f'../visualization/dynamics/grid_plots/experiment_{experiment}.png')

### Experiment 1

In [ ]:
experiment_number = 1
INFO = """1990 to 2020 every 4 month, 4-years windows,
    thresholds [-0.1; -2.0],
    2000 bins"""
RUN_IDS = {
    'UN': '003865Y',
    'UW': '643630M',
    'LN': '653851X',
    'JT': '102939K',
}
data = pd.DataFrame()
for market, run_id in RUN_IDS.items():
    df = read_pickle(run_id, market)
    data = pd.concat([data, df])
# Plot Seaborn FacetGrid
g = plot_indicators_grid(data)
g.figure.suptitle(INFO, fontsize=20)
plt.tight_layout()
plt.subplots_adjust(top=0.88)
save_figure(g, experiment_number)
plt.show()

### Experiment 2

In [ ]:
experiment_number = 2
INFO = """1990 to 2020 every 4 month, 2-years windows,
    thresholds [-0.1; -2.0],
    2000 bins"""
RUN_IDS = {
    'UN': '365374G',
    'UW': '655972O',
    'LN': '941047N',
    'JT': '019647Q',
}
data = pd.DataFrame()
for market, run_id in RUN_IDS.items():
    df = read_pickle(run_id, market)
    data = pd.concat([data, df])
# Plot Seaborn FacetGrid
g = plot_indicators_grid(data)
g.figure.suptitle(INFO, fontsize=20)
plt.tight_layout()
plt.subplots_adjust(top=0.88)
save_figure(g, experiment_number)
plt.show()

### Experiment 3

In [ ]:
experiment_number = 3
INFO = """1990 to 2020 every 4 month, 1-year windows,
    thresholds [-0.1; -2.0],
    2000 bins"""
RUN_IDS = {
    'UN': '928387S',
    'UW': '774892E',
    'LN': '960223U',
    'JT': '897643T',
}
data = pd.DataFrame()
for market, run_id in RUN_IDS.items():
    df = read_pickle(run_id, market)
    data = pd.concat([data, df])
# Plot Seaborn FacetGrid
g = plot_indicators_grid(data)
g.figure.suptitle(INFO, fontsize=20)
plt.tight_layout()
plt.subplots_adjust(top=0.88)
save_figure(g, experiment_number)
plt.show()

### Experiment 4

In [ ]:
experiment_number = 4
INFO = """1990 to 2020 every 4 month, 1-year windows,
    thresholds [-0.1; -2.0],
    1000 bins"""
RUN_IDS = {
    'UN': '242769F',
    'UW': '963882T',
    'LN': '088988V',
    'JT': '711534E',
}
data = pd.DataFrame()
for market, run_id in RUN_IDS.items():
    df = read_pickle(run_id, market)
    data = pd.concat([data, df])
# Plot Seaborn FacetGrid
g = plot_indicators_grid(data)
g.figure.suptitle(INFO, fontsize=20)
plt.tight_layout()
plt.subplots_adjust(top=0.88)
save_figure(g, experiment_number)
plt.show()

### Experiment 5

In [ ]:
experiment_number = 5
INFO = """1990 to 2020 every 4 month, 1-year windows,
    thresholds [-0.1; -2.0],
    500 bins"""
RUN_IDS = {
    'UN': '565988I',
    'UW': '899299O',
    'LN': '368353O',
    'JT': '668904O',
}
data = pd.DataFrame()
for market, run_id in RUN_IDS.items():
    df = read_pickle(run_id, market)
    data = pd.concat([data, df])
# Plot Seaborn FacetGrid
g = plot_indicators_grid(data)
g.figure.suptitle(INFO, fontsize=20)
plt.tight_layout()
plt.subplots_adjust(top=0.88)
save_figure(g, experiment_number)
plt.show()

### Experiment 6

In [ ]:
experiment_number = 6
INFO = """1990 to 2020 every 4 month, 2-years windows,
    thresholds [-0.1; -1.5],
    2000 bins"""
RUN_IDS = {
    'UN': '852031O',
    'UW': '605850U',
    'LN': '021340F',
    'JT': '482288K',
}
data = pd.DataFrame()
for market, run_id in RUN_IDS.items():
    df = read_pickle(run_id, market)
    data = pd.concat([data, df])
# Plot Seaborn FacetGrid
g = plot_indicators_grid(data)
g.figure.suptitle(INFO, fontsize=20)
plt.tight_layout()
plt.subplots_adjust(top=0.88)
save_figure(g, experiment_number)
plt.show()

### Experiment 7

In [ ]:
experiment_number = 7
INFO = """1990 to 2020 every 4 month, 4-years windows,
    thresholds [-0.1; -1.5],
    2000 bins"""
RUN_IDS = {
    'UN': '242914S',
    'UW': '565750N',
    'LN': '429252I',
    'JT': '943908F',
}
data = pd.DataFrame()
for market, run_id in RUN_IDS.items():
    df = read_pickle(run_id, market)
    data = pd.concat([data, df])
# Plot Seaborn FacetGrid
g = plot_indicators_grid(data)
g.figure.suptitle(INFO, fontsize=20)
plt.tight_layout()
plt.subplots_adjust(top=0.88)
save_figure(g, experiment_number)
plt.show()

### Experiment 8

In [ ]:
experiment_number = 8
INFO = """1990 to 2020 every 4 month, 4-years windows,
    thresholds [-0.1; -1.5],
    5000 bins"""
RUN_IDS = {
    'UN': '709338D',
    'UW': '927491K',
    'LN': '289657I',
    'JT': '902600X',
}
data = pd.DataFrame()
for market, run_id in RUN_IDS.items():
    df = read_pickle(run_id, market)
    data = pd.concat([data, df])
# Plot Seaborn FacetGrid
g = plot_indicators_grid(data)
g.figure.suptitle(INFO, fontsize=20)
plt.tight_layout()
plt.subplots_adjust(top=0.88)
save_figure(g, experiment_number)
plt.show()

### Experiment 9

In [ ]:
experiment_number = 9
INFO = """1990 to 2020 every 4 month, 4-years windows,
    thresholds [-0.1; -1.5],
    500 bins"""
RUN_IDS = {
    'UN': '042109Z',
    'UW': '715743X',
    'LN': '062485K',
    'JT': '927218C',
}
data = pd.DataFrame()
for market, run_id in RUN_IDS.items():
    df = read_pickle(run_id, market)
    data = pd.concat([data, df])
# Plot Seaborn FacetGrid
g = plot_indicators_grid(data)
g.figure.suptitle(INFO, fontsize=20)
plt.tight_layout()
plt.subplots_adjust(top=0.88)
save_figure(g, experiment_number)
plt.show()

### Experiment 10

In [ ]:
experiment_number = 10
INFO = """1990 to 2020 every 4 month, 2-years windows,
    thresholds [-0.1; -1.5],
    500 bins"""
RUN_IDS = {
    'UN': '855945R',
    'UW': '877537K',
    'LN': '775312L',
    'JT': '637990M',
}
data = pd.DataFrame()
for market, run_id in RUN_IDS.items():
    df = read_pickle(run_id, market)
    data = pd.concat([data, df])
# Plot Seaborn FacetGrid
g = plot_indicators_grid(data)
g.figure.suptitle(INFO, fontsize=20)
plt.tight_layout()
plt.subplots_adjust(top=0.88)
save_figure(g, experiment_number)
plt.show()

### Experiment 11

In [ ]:
experiment_number = 11
INFO = """1990 to 2020 every 2 months, 2-years windows,
    thresholds [-0.1; -2.0],
    2000 bins"""
RUN_IDS = {
    'UN': '720192X',
    'UW': '811006M',
    'LN': '739420C',
    'JT': '223400A',
}
data = pd.DataFrame()
for market, run_id in RUN_IDS.items():
    df = read_pickle(run_id, market)
    data = pd.concat([data, df])
# Plot Seaborn FacetGrid
g = plot_indicators_grid(data)
g.figure.suptitle(INFO, fontsize=20)
plt.tight_layout()
plt.subplots_adjust(top=0.88)
save_figure(g, experiment_number)
plt.show()

### Experiment 12

In [ ]:
experiment_number = 12
INFO = """1990 to 2020 every 2 months, 2-years windows,
    thresholds [0.0; -1.0],
    1000 bins"""
RUN_IDS = {
    'UN': '329900I',
    'UW': '971711C',
    'LN': '129109Y',
    'JT': '864193Z',
}
data = pd.DataFrame()
for market, run_id in RUN_IDS.items():
    df = read_pickle(run_id, market)
    data = pd.concat([data, df])
# Plot Seaborn FacetGrid
g = plot_indicators_grid(data)
g.figure.suptitle(INFO, fontsize=20)
plt.tight_layout()
plt.subplots_adjust(top=0.88)
save_figure(g, experiment_number)
plt.show()

### Experiment 13

In [ ]:
experiment_number = 13
INFO = """1990 to 2020 every 2 months, 2-years windows,
    thresholds [-0.1; -2.0],
    1000 bins"""
RUN_IDS = {
    'UN': '331719B',
    'UW': '927049F',
    'LN': '222677X',
    'JT': '737397X',
}
data = pd.DataFrame()
for market, run_id in RUN_IDS.items():
    df = read_pickle(run_id, market)
    data = pd.concat([data, df])
# Plot Seaborn FacetGrid
g = plot_indicators_grid(data)
g.figure.suptitle(INFO, fontsize=20)
plt.tight_layout()
plt.subplots_adjust(top=0.88)
save_figure(g, experiment_number)
plt.show()

### Experiment 14

In [ ]:
experiment_number = 14
INFO = """1990 to 2020 every 2 months, 1-years windows,
    thresholds [0.0; -1.0],
    1000 bins"""
RUN_IDS = {
    'UN': '992991E',
    'UW': '883178B',
    'LN': '225521B',
    'JT': '091567J',
}
data = pd.DataFrame()
for market, run_id in RUN_IDS.items():
    df = read_pickle(run_id, market)
    data = pd.concat([data, df])
# Plot Seaborn FacetGrid
g = plot_indicators_grid(data)
g.figure.suptitle(INFO, fontsize=20)
plt.tight_layout()
plt.subplots_adjust(top=0.88)
save_figure(g, experiment_number)
plt.show()

### Experiment 15

In [ ]:
experiment_number = 15
INFO = """1990 to 2020 every 2 months, 1-years windows,
    thresholds [-0.1; -2.5],
    1000 bins"""
RUN_IDS = {
    'UN': '107340Q',
    'UW': '258743T',
    'LN': '926899H',
    'JT': '781892G',
}
data = pd.DataFrame()
for market, run_id in RUN_IDS.items():
    df = read_pickle(run_id, market)
    data = pd.concat([data, df])
# Plot Seaborn FacetGrid
g = plot_indicators_grid(data)
g.figure.suptitle(INFO, fontsize=20)
plt.tight_layout()
plt.subplots_adjust(top=0.88)
save_figure(g, experiment_number)
plt.show()

### 4 Months Steps - 4 Years Windows

In [ ]:
data = pd.DataFrame()

for rolling_chunk, experiments in ROLLING_CHUNKS.items():
    for n in experiments:
        for market, run_id in  EXPERIMENTS[n-1].items():
            df = read_pickle(run_id, market)
            df['Thresholds'] = ROLLING_FREE_PARAMETERS[n][0]
            df['Bins'] = ROLLING_FREE_PARAMETERS[n][1]
            df['Rolling'] = rolling_chunk
            data = pd.concat([data, df])
data = data.reset_index()
data.describe()

### Superplots Plot

In [ ]:
def superplot(variable):
    g = sns.relplot(data,
                    x='Day', y=variable,
                    hue='Thresholds',
                    size='Bins',
                    col='Market',
                    kind='line',
                    aspect=1.3,
                    )
    g.map(lambda **kwargs: plt.axvline(milestone1, color='green', linestyle='--'))
    g.map(lambda **kwargs: plt.axvline(milestone2, color='red', linestyle='--'))
    return g

for variable in ['Max', 'Peak', 'FWHM']:
    g = superplot(variable)
    g.savefig(f'../visualization/dynamics/superplots/{variable}.png')
    plt.show()

In [ ]:
variable = 'Peak'
g = superplot(variable)
g.axes[0][0].set_ylim(0.0, 0.025)
g.savefig(f'../visualization/dynamics/superplots/{variable}_zoom.png')
plt.show()

In [ ]:
variable = 'FWHM'
g = superplot(variable)
g.axes[0][0].set_ylim(-0.001, 0.022)
g.savefig(f'../visualization/dynamics/superplots/{variable}_zoom.png')
plt.show()